In [206]:
import pandas as pd
import sys
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import math
from math import sqrt
sys.path.insert(0,"../")

In [207]:
import logging

from tqdm.notebook import tqdm
tqdm.pandas()

logger = logging.getLogger(__name__)

logger.setLevel(logging.CRITICAL)

import warnings; warnings.simplefilter('ignore')


# Read in predictions data + true close 

In [506]:
coin_predictions_df = pd.read_csv("training_configs/eth_all_predictions.csv")
true_price_df = pd.read_csv("../tmp/historic_crypto_prices - etherum_jan_2017_sept_4_2021 copy.csv")

In [507]:
coin_predictions_df.head()

,date,nbeats_eth_lookback_15_window_5_std_1.25_num_add_dfs_2,tcn_eth_lookback_15_window_5_std_1.25_num_add_dfs_2,nbeats_eth_lookback_30_window_5_std_1.25_num_add_dfs_2,tcn_eth_lookback_30_window_5_std_1.25_num_add_dfs_2,nbeats_eth_lookback_45_window_5_std_1.25_num_add_dfs_2,tcn_eth_lookback_45_window_5_std_1.25_num_add_dfs_2,date_prediction_for,test_model_lookback_1
0,2010-01-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
1,2019-04-02,160.726046,108.336161,162.136892,4.388280,157.352095,38.469269,2019-04-09,0.0
2,2019-04-03,156.490182,43.245200,156.719728,10.110944,144.199683,112.081314,2019-04-10,0.0
3,2019-04-04,164.944170,96.866299,150.058959,9.733006,167.251629,24.725918,2019-04-11,0.0
4,2019-04-05,169.581252,20.152966,142.574369,2.398744,152.488238,24.184747,2019-04-12,0.0


In [508]:
coin_predictions_df.tail()

,date,nbeats_eth_lookback_15_window_5_std_1.25_num_add_dfs_2,tcn_eth_lookback_15_window_5_std_1.25_num_add_dfs_2,nbeats_eth_lookback_30_window_5_std_1.25_num_add_dfs_2,tcn_eth_lookback_30_window_5_std_1.25_num_add_dfs_2,nbeats_eth_lookback_45_window_5_std_1.25_num_add_dfs_2,tcn_eth_lookback_45_window_5_std_1.25_num_add_dfs_2,date_prediction_for,test_model_lookback_1
1097,2022-04-02,3535.769389,3175.882685,3255.820443,3200.570144,2786.853463,3207.790335,2022-04-09,0.0
1098,2022-04-03,3234.399281,3505.240524,3343.657822,3445.156287,3256.887045,3372.260822,2022-04-10,0.0
1099,2022-04-04,3105.531731,3356.971676,3088.940826,3242.422896,3384.488514,3188.002936,2022-04-11,0.0
1100,2022-04-05,3575.013311,3328.596442,3187.696664,3305.310063,3449.950517,3254.584578,2022-04-12,0.0
1101,2022-04-06,3258.340999,3127.607504,3376.711133,3244.755386,3527.194692,3312.358373,2022-04-13,0.0


In [509]:
true_price_df.head()

,date,open,high,low,close,volume
0,2017-01-01,7.98,8.47,7.98,8.17,14731700.0
1,2017-01-02,8.17,8.44,8.05,8.38,14579600.0
2,2017-01-03,8.37,10.00,8.32,9.73,33625200.0
3,2017-01-04,9.71,11.28,9.56,11.25,41051200.0
4,2017-01-05,11.29,11.89,9.40,10.25,41557400.0


- merged true close price and prediction

In [549]:
true_price_df = true_price_df[['date','close']]
merged_df = pd.merge(coin_predictions_df, true_price_df, left_on='date_prediction_for',
                     right_on ='date', suffixes=['_pred','_true'])

In [550]:
merged_df['day'] = [t.day for t in pd.to_datetime(merged_df.date_prediction_for)]
merged_df['month'] =  [t.month for t in pd.to_datetime(merged_df.date_prediction_for)]
merged_df['quarter'] = [t.quarter for t in pd.to_datetime(merged_df.date_prediction_for)]
merged_df['day_of_year'] = [t.strftime('%j') for t in pd.to_datetime(merged_df.date_prediction_for)]
merged_df['year'] = [t.year for t in pd.to_datetime(merged_df.date_prediction_for)]

In [551]:
true_price_df['day'] = [t.day for t in pd.to_datetime(true_price_df.date)]
true_price_df['month'] =  [t.month for t in pd.to_datetime(true_price_df.date)]
true_price_df['quarter'] = [t.quarter for t in pd.to_datetime(true_price_df.date)]
true_price_df['day_of_year'] = [t.strftime('%j') for t in pd.to_datetime(true_price_df.date)]
true_price_df['year'] = [t.year for t in pd.to_datetime(true_price_df.date)]

In [552]:
true_price_df

,date,close,day,month,quarter,day_of_year,year
0,2017-01-01,8.17,1,1,1,001,2017
1,2017-01-02,8.38,2,1,1,002,2017
2,2017-01-03,9.73,3,1,1,003,2017
3,2017-01-04,11.25,4,1,1,004,2017
4,2017-01-05,10.25,5,1,1,005,2017
...,...,...,...,...,...,...,...
1921,2022-04-06,3167.60,6,4,2,096,2022
1922,2022-04-07,3227.10,7,4,2,097,2022
1923,2022-04-08,3193.40,8,4,2,098,2022
1924,2022-04-09,3258.90,9,4,2,099,2022


In [553]:
merged_df

,date_pred,nbeats_eth_lookback_15_window_5_std_1.25_num_add_dfs_2,tcn_eth_lookback_15_window_5_std_1.25_num_add_dfs_2,nbeats_eth_lookback_30_window_5_std_1.25_num_add_dfs_2,tcn_eth_lookback_30_window_5_std_1.25_num_add_dfs_2,nbeats_eth_lookback_45_window_5_std_1.25_num_add_dfs_2,tcn_eth_lookback_45_window_5_std_1.25_num_add_dfs_2,date_prediction_for,test_model_lookback_1,date_true,close,day,month,quarter,day_of_year,year
0,2019-04-02,160.726046,108.336161,162.136892,4.388280,157.352095,38.469269,2019-04-09,0.0,2019-04-09,176.11,9,4,2,099,2019
1,2019-04-03,156.490182,43.245200,156.719728,10.110944,144.199683,112.081314,2019-04-10,0.0,2019-04-10,177.34,10,4,2,100,2019
2,2019-04-04,164.944170,96.866299,150.058959,9.733006,167.251629,24.725918,2019-04-11,0.0,2019-04-11,165.50,11,4,2,101,2019
3,2019-04-05,169.581252,20.152966,142.574369,2.398744,152.488238,24.184747,2019-04-12,0.0,2019-04-12,164.74,12,4,2,102,2019
4,2019-04-06,168.788176,25.307259,168.871998,48.561893,178.562880,109.212879,2019-04-13,0.0,2019-04-13,164.51,13,4,2,103,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1093,2022-03-30,3306.404151,3023.395365,3727.543705,3117.324561,3220.064806,2726.300653,2022-04-06,0.0,2022-04-06,3167.60,6,4,2,096,2022
1094,2022-03-31,3196.371094,3080.311473,3271.641192,2981.440724,3077.196131,3032.012491,2022-04-07,0.0,2022-04-07,3227.10,7,4,2,097,2022
1095,2022-04-01,3226.660704,3331.778384,3460.590677,3226.893086,3401.522444,3244.867382,2022-04-08,0.0,2022-04-08,3193.40,8,4,2,098,2022
1096,2022-04-02,3535.769389,3175.882685,3255.820443,3200.570144,2786.853463,3207.790335,2022-04-09,0.0,2022-04-09,3258.90,9,4,2,099,2022


In [514]:
dropped_merged_df = merged_df.drop(['date_pred', 'date_true', 'date_prediction_for'],  axis=1)

# Test the different algos
- mean
- RF
- GB
- LR

In [515]:
dropped_merged_df

,nbeats_eth_lookback_15_window_5_std_1.25_num_add_dfs_2,tcn_eth_lookback_15_window_5_std_1.25_num_add_dfs_2,nbeats_eth_lookback_30_window_5_std_1.25_num_add_dfs_2,tcn_eth_lookback_30_window_5_std_1.25_num_add_dfs_2,nbeats_eth_lookback_45_window_5_std_1.25_num_add_dfs_2,tcn_eth_lookback_45_window_5_std_1.25_num_add_dfs_2,test_model_lookback_1,close,day,month,quarter,day_of_year,year
0,160.726046,108.336161,162.136892,4.388280,157.352095,38.469269,0.0,176.11,9,4,2,099,2019
1,156.490182,43.245200,156.719728,10.110944,144.199683,112.081314,0.0,177.34,10,4,2,100,2019
2,164.944170,96.866299,150.058959,9.733006,167.251629,24.725918,0.0,165.50,11,4,2,101,2019
3,169.581252,20.152966,142.574369,2.398744,152.488238,24.184747,0.0,164.74,12,4,2,102,2019
4,168.788176,25.307259,168.871998,48.561893,178.562880,109.212879,0.0,164.51,13,4,2,103,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1093,3306.404151,3023.395365,3727.543705,3117.324561,3220.064806,2726.300653,0.0,3167.60,6,4,2,096,2022
1094,3196.371094,3080.311473,3271.641192,2981.440724,3077.196131,3032.012491,0.0,3227.10,7,4,2,097,2022
1095,3226.660704,3331.778384,3460.590677,3226.893086,3401.522444,3244.867382,0.0,3193.40,8,4,2,098,2022
1096,3535.769389,3175.882685,3255.820443,3200.570144,2786.853463,3207.790335,0.0,3258.90,9,4,2,099,2022


In [516]:
merged_df

,date_pred,nbeats_eth_lookback_15_window_5_std_1.25_num_add_dfs_2,tcn_eth_lookback_15_window_5_std_1.25_num_add_dfs_2,nbeats_eth_lookback_30_window_5_std_1.25_num_add_dfs_2,tcn_eth_lookback_30_window_5_std_1.25_num_add_dfs_2,nbeats_eth_lookback_45_window_5_std_1.25_num_add_dfs_2,tcn_eth_lookback_45_window_5_std_1.25_num_add_dfs_2,date_prediction_for,test_model_lookback_1,date_true,close,day,month,quarter,day_of_year,year
0,2019-04-02,160.726046,108.336161,162.136892,4.388280,157.352095,38.469269,2019-04-09,0.0,2019-04-09,176.11,9,4,2,099,2019
1,2019-04-03,156.490182,43.245200,156.719728,10.110944,144.199683,112.081314,2019-04-10,0.0,2019-04-10,177.34,10,4,2,100,2019
2,2019-04-04,164.944170,96.866299,150.058959,9.733006,167.251629,24.725918,2019-04-11,0.0,2019-04-11,165.50,11,4,2,101,2019
3,2019-04-05,169.581252,20.152966,142.574369,2.398744,152.488238,24.184747,2019-04-12,0.0,2019-04-12,164.74,12,4,2,102,2019
4,2019-04-06,168.788176,25.307259,168.871998,48.561893,178.562880,109.212879,2019-04-13,0.0,2019-04-13,164.51,13,4,2,103,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1093,2022-03-30,3306.404151,3023.395365,3727.543705,3117.324561,3220.064806,2726.300653,2022-04-06,0.0,2022-04-06,3167.60,6,4,2,096,2022
1094,2022-03-31,3196.371094,3080.311473,3271.641192,2981.440724,3077.196131,3032.012491,2022-04-07,0.0,2022-04-07,3227.10,7,4,2,097,2022
1095,2022-04-01,3226.660704,3331.778384,3460.590677,3226.893086,3401.522444,3244.867382,2022-04-08,0.0,2022-04-08,3193.40,8,4,2,098,2022
1096,2022-04-02,3535.769389,3175.882685,3255.820443,3200.570144,2786.853463,3207.790335,2022-04-09,0.0,2022-04-09,3258.90,9,4,2,099,2022


## Compare different algos
- Test each day individually

In [517]:
COLS_TO_EXCLUDE = ['test_model_lookback_1', 'close','rolling_mean','bollinger_high','bollinger_low', 'date', 'date_pred', 'date_true', 'date_prediction_for']
DAY_PART_COLS = ['day', 'month', 'quarter', 'day_of_year', 'year']


In [518]:
gb_pred_list = []
rf_pred_list = []
lr_pred_list = []
mean_pred_list = []
actual_close_list = []


for row_idx, row in enumerate(dropped_merged_df.iterrows()):
    if row_idx == 0: # need one day to start training
        continue
    
    if row_idx % 10 == 0:
        print(f"{row_idx/len(dropped_merged_df)*100:.2f}%")

    gb = GradientBoostingRegressor(n_estimators=150)
    rf = RandomForestRegressor(n_estimators=200, n_jobs=-1)
    lr = LinearRegression()
    
    
    stacking_training_cols = row[1].index[~row[1].index.isin(COLS_TO_EXCLUDE)]
    stacked_x_data = dropped_merged_df[stacking_training_cols][:row_idx] # don't include the current row
    stacked_y_data = dropped_merged_df[PRED_COL][:row_idx] # don't include the current row
    
    gb.fit(stacked_x_data, stacked_y_data)
    gb_pred = gb.predict(np.array(dropped_merged_df[stacking_training_cols].iloc[row_idx]).reshape(1,-1))
    gb_pred_list.append(gb_pred[0])
    
    rf.fit(stacked_x_data, stacked_y_data)
    rf_pred = rf.predict(np.array(dropped_merged_df[stacking_training_cols].iloc[row_idx]).reshape(1,-1))
    rf_pred_list.append(rf_pred[0])
    
    lr.fit(stacked_x_data, stacked_y_data)
    lr_pred = lr.predict(np.array(dropped_merged_df[stacking_training_cols].iloc[row_idx]).reshape(1,-1))
    lr_pred_list.append(lr_pred[0])
    
    
    mean_pred_cols = row[1].index[~row[1].index.isin(COLS_TO_EXCLUDE+DAY_PART_COLS)]
    mean_pred = np.mean(row[1][mean_pred_cols])
    mean_pred_list.append(mean_pred)
    
    actual_close_list.append(row[1][PRED_COL])


0.91%
1.82%
2.73%
3.64%
4.55%
5.46%
6.38%
7.29%
8.20%
9.11%
10.02%
10.93%
11.84%
12.75%
13.66%
14.57%
15.48%
16.39%
17.30%
18.21%
19.13%
20.04%
20.95%
21.86%
22.77%
23.68%
24.59%
25.50%
26.41%
27.32%
28.23%
29.14%
30.05%
30.97%
31.88%
32.79%
33.70%
34.61%
35.52%
36.43%
37.34%
38.25%
39.16%
40.07%
40.98%
41.89%
42.81%
43.72%
44.63%
45.54%
46.45%
47.36%
48.27%
49.18%
50.09%
51.00%
51.91%
52.82%
53.73%
54.64%
55.56%
56.47%
57.38%
58.29%
59.20%
60.11%
61.02%
61.93%
62.84%
63.75%
64.66%
65.57%
66.48%
67.40%
68.31%
69.22%
70.13%
71.04%
71.95%
72.86%
73.77%
74.68%
75.59%
76.50%
77.41%
78.32%
79.23%
80.15%
81.06%
81.97%
82.88%
83.79%
84.70%
85.61%
86.52%
87.43%
88.34%
89.25%
90.16%
91.07%
91.99%
92.90%
93.81%
94.72%
95.63%
96.54%
97.45%
98.36%
99.27%


In [520]:
rf_pred_list[-10:]

[3344.133349999999,
 3061.8067500000006,
 3511.158950000001,
 3547.37475,
 3503.5463999999997,
 3438.247950000002,
 3375.4314499999978,
 3352.8010999999997,
 3297.113400000002,
 3278.794050000006]

In [521]:
mean_pred_list[-10:]

[3097.047521835335,
 2922.975146276605,
 2973.334029300879,
 3123.329278127687,
 3141.666594446457,
 3186.8388736574057,
 3106.495517337775,
 3315.385446233708,
 3193.781076520019,
 3359.600296885203]

In [355]:
gb_pred_list[-10:]

[40675.71370974031,
 45381.75167187664,
 45120.88854813461,
 45240.09422739998,
 43811.41316373885,
 46345.95721722673,
 45222.97332554667,
 47830.295333339265,
 50074.177461083906,
 48989.03441606732]

In [356]:
rf_pred_list[-10:]

[43030.93955,
 44504.3692,
 43447.077750000004,
 44411.49779999999,
 42610.78920000001,
 44075.0976,
 44448.9092,
 46234.079349999985,
 47424.94089999999,
 45871.2632]

In [357]:
actual_close_list[-10:]

[47160.0,
 47456.0,
 47085.0,
 45520.0,
 46300.0,
 45817.0,
 46405.0,
 46592.0,
 45504.0,
 43166.0]

In [358]:
dropped_merged_df["nbeats_btc_lookback_15_window_5_std_1.25_num_add_dfs_2"]

date_true
2019-04-09     4752.149527
2019-04-10     4546.248123
2019-04-11     4550.998590
2019-04-12     5003.264197
2019-04-13     5184.268504
                  ...     
2022-04-02    43746.323175
2022-04-03    43266.796507
2022-04-04    44123.399870
2022-04-05    48795.983121
2022-04-06    46296.230360
Name: nbeats_btc_lookback_15_window_5_std_1.25_num_add_dfs_2, Length: 1094, dtype: float64

In [522]:
dropped_merged_df

,nbeats_eth_lookback_15_window_5_std_1.25_num_add_dfs_2,tcn_eth_lookback_15_window_5_std_1.25_num_add_dfs_2,nbeats_eth_lookback_30_window_5_std_1.25_num_add_dfs_2,tcn_eth_lookback_30_window_5_std_1.25_num_add_dfs_2,nbeats_eth_lookback_45_window_5_std_1.25_num_add_dfs_2,tcn_eth_lookback_45_window_5_std_1.25_num_add_dfs_2,test_model_lookback_1,close,day,month,quarter,day_of_year,year
0,160.726046,108.336161,162.136892,4.388280,157.352095,38.469269,0.0,176.11,9,4,2,099,2019
1,156.490182,43.245200,156.719728,10.110944,144.199683,112.081314,0.0,177.34,10,4,2,100,2019
2,164.944170,96.866299,150.058959,9.733006,167.251629,24.725918,0.0,165.50,11,4,2,101,2019
3,169.581252,20.152966,142.574369,2.398744,152.488238,24.184747,0.0,164.74,12,4,2,102,2019
4,168.788176,25.307259,168.871998,48.561893,178.562880,109.212879,0.0,164.51,13,4,2,103,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1093,3306.404151,3023.395365,3727.543705,3117.324561,3220.064806,2726.300653,0.0,3167.60,6,4,2,096,2022
1094,3196.371094,3080.311473,3271.641192,2981.440724,3077.196131,3032.012491,0.0,3227.10,7,4,2,097,2022
1095,3226.660704,3331.778384,3460.590677,3226.893086,3401.522444,3244.867382,0.0,3193.40,8,4,2,098,2022
1096,3535.769389,3175.882685,3255.820443,3200.570144,2786.853463,3207.790335,0.0,3258.90,9,4,2,099,2022


In [523]:
rmse_nbeats_15 = sqrt(mean_squared_error(dropped_merged_df["nbeats_eth_lookback_15_window_5_std_1.25_num_add_dfs_2"], dropped_merged_df.close))
rmse_nbeats_30 = sqrt(mean_squared_error(dropped_merged_df["nbeats_eth_lookback_30_window_5_std_1.25_num_add_dfs_2"], dropped_merged_df.close))
rmse_nbeats_45 = sqrt(mean_squared_error(dropped_merged_df["nbeats_eth_lookback_45_window_5_std_1.25_num_add_dfs_2"], dropped_merged_df.close))

rmse_tcn_15 = sqrt(mean_squared_error(dropped_merged_df["tcn_eth_lookback_15_window_5_std_1.25_num_add_dfs_2"], dropped_merged_df.close))
rmse_tcn_30 = sqrt(mean_squared_error(dropped_merged_df["tcn_eth_lookback_30_window_5_std_1.25_num_add_dfs_2"], dropped_merged_df.close))
rmse_tcn_45 = sqrt(mean_squared_error(dropped_merged_df["tcn_eth_lookback_45_window_5_std_1.25_num_add_dfs_2"], dropped_merged_df.close))

## ETH results

In [524]:
rmse_gb = sqrt(mean_squared_error(gb_pred_list, actual_close_list))
rmse_rf = sqrt(mean_squared_error(rf_pred_list, actual_close_list))
rmse_lr = sqrt(mean_squared_error(lr_pred_list, actual_close_list))
rmse_average = sqrt(mean_squared_error(mean_pred_list, actual_close_list))

print(f"rmse_gb = {rmse_gb}")
print(f"rmse_rf = {rmse_rf}")
print(f"rmse_lr = {rmse_lr}")
print(f"rmse_average = {rmse_average}")
print(f" rmse_nbeats_15 rmse ={rmse_nbeats_15}")
print(f" rmse_nbeats_30 rmse ={rmse_nbeats_30}")
print(f" rmse_nbeats_45 rmse ={rmse_nbeats_45}")
print(f" rmse_tcn_15 rmse ={rmse_tcn_15}")
print(f" rmse_tcn_30 rmse ={rmse_tcn_30}")
print(f" rmse_tcn_45 rmse ={rmse_tcn_45}")

rmse_gb = 145.63460261093726
rmse_rf = 162.76576877866742
rmse_lr = 264.8809643429884
rmse_average = 294.13635991249447
 rmse_nbeats_15 rmse =335.8897244909966
 rmse_nbeats_30 rmse =341.55014842810596
 rmse_nbeats_45 rmse =385.22370153633267
 rmse_tcn_15 rmse =286.3391951102119
 rmse_tcn_30 rmse =302.9607736306262
 rmse_tcn_45 rmse =330.97838047724116


In [525]:
145/162 # GB is 10% better than RF

0.8950617283950617

In [527]:
145/294 # GB is 50% better than mean

0.4931972789115646

## BTC results

In [360]:
rmse_gb = sqrt(mean_squared_error(gb_pred_list, actual_close_list))
rmse_rf = sqrt(mean_squared_error(rf_pred_list, actual_close_list))
rmse_lr = sqrt(mean_squared_error(lr_pred_list, actual_close_list))
rmse_average = sqrt(mean_squared_error(mean_pred_list, actual_close_list))

print(f"rmse_gb = {rmse_gb}")
print(f"rmse_rf = {rmse_rf}")
print(f"rmse_lr = {rmse_lr}")
print(f"rmse_average = {rmse_average}")
print(f" rmse_nbeats_15 rmse ={rmse_nbeats_15}")
print(f" rmse_nbeats_30 rmse ={rmse_nbeats_30}")
print(f" rmse_nbeats_45 rmse ={rmse_nbeats_45}")
print(f" rmse_tcn_15 rmse ={rmse_tcn_15}")
print(f" rmse_tcn_30 rmse ={rmse_tcn_30}")
print(f" rmse_tcn_45 rmse ={rmse_tcn_45}")

rmse_gb = 2033.4112310145047
rmse_rf = 1976.2592556300715
rmse_lr = 3714.8071439413316
rmse_average = 3900.521632858845
 rmse_nbeats_15 rmse =4155.783424507883
 rmse_nbeats_30 rmse =4516.634674510583
 rmse_nbeats_45 rmse =4805.890352853746
 rmse_tcn_15 rmse =4482.577636950043
 rmse_tcn_30 rmse =4457.042411775085
 rmse_tcn_45 rmse =4817.038189648862


In [361]:
rmse_rf/rmse_gb-1 # 5% better rmse = 2048 RF w/o year, 1965 with year , 4% better with year

-0.028106452109994073

In [362]:
rmse_rf/rmse_average-1 # lol 52% better

-0.49333462504562653

# Test determine trading state strategy


In [528]:
def _determine_win_or_loss_amount(row, mode, buy_entry_price, todays_close_price, position_entry_date, 
                                  n_buy_lost, n_buy_won, dollar_amount_buy_won, dollar_amount_buy_lost, n_total_days_in_trades):
       #self._determine_win_or_loss_amount(row)
    # lost money
    if mode == "buy" and buy_entry_price > todays_close_price:

        lost_amount = buy_entry_price - todays_close_price
        print(f"Lost {lost_amount} on this trade")

        n_buy_lost += 1
        dollar_amount_buy_lost += lost_amount

    # made money
    elif mode == "buy" and buy_entry_price < todays_close_price:

        won_amount = todays_close_price - buy_entry_price
        print(f"Won {won_amount} on this trade")
        n_buy_won += 1
        dollar_amount_buy_won += won_amount
    else:
        raise ValueError("Something went wrong calculating win/lose amount")

    days_in_trade = pd.to_datetime(row.name) - pd.to_datetime(position_entry_date)
    print(f"days in trades = {days_in_trade.days}")
    n_total_days_in_trades += days_in_trade.days

    # info logging

    print(
        f"Average days in trades = {n_total_days_in_trades/(n_buy_won  + n_buy_lost )}"
    )
    if n_buy_won > 0 or n_buy_lost > 0:
        print(f"Bat rate buy so far = {n_buy_won / (n_buy_won + n_buy_lost)}")


    if dollar_amount_buy_won > 0 or dollar_amount_buy_lost > 0:
        logger.info(
            f"Win rate buy so far = {float(dollar_amount_buy_won) / float(float(dollar_amount_buy_won) + float(dollar_amount_buy_lost))}"
        )


    print(f"Total days in trades = {n_total_days_in_trades }")

    # reset for sanity
    position_entry_date = None
        
    return mode, buy_entry_price, todays_close_price, position_entry_date, n_buy_lost, n_buy_won, dollar_amount_buy_won, dollar_amount_buy_lost, n_total_days_in_trades
        

- create new DF where the dates are the actual date of the close. Predicting 7 days into the future for prices

In [535]:
true_price_df

,date,close,day,month,quarter,day_of_year,year
0,2017-01-01,8.17,1,1,1,001,2017
1,2017-01-02,8.38,2,1,1,002,2017
2,2017-01-03,9.73,3,1,1,003,2017
3,2017-01-04,11.25,4,1,1,004,2017
4,2017-01-05,10.25,5,1,1,005,2017
...,...,...,...,...,...,...,...
1921,2022-04-06,3167.60,6,4,2,096,2022
1922,2022-04-07,3227.10,7,4,2,097,2022
1923,2022-04-08,3193.40,8,4,2,098,2022
1924,2022-04-09,3258.90,9,4,2,099,2022


In [536]:
true_price_df

,date,close,day,month,quarter,day_of_year,year
0,2017-01-01,8.17,1,1,1,001,2017
1,2017-01-02,8.38,2,1,1,002,2017
2,2017-01-03,9.73,3,1,1,003,2017
3,2017-01-04,11.25,4,1,1,004,2017
4,2017-01-05,10.25,5,1,1,005,2017
...,...,...,...,...,...,...,...
1921,2022-04-06,3167.60,6,4,2,096,2022
1922,2022-04-07,3227.10,7,4,2,097,2022
1923,2022-04-08,3193.40,8,4,2,098,2022
1924,2022-04-09,3258.90,9,4,2,099,2022


In [539]:
merged_df_determine_trading_state = pd.merge(coin_predictions_df, true_price_df,
                                             left_on='date', right_on='date', suffixes=['_pred','_true'])


In [540]:
merged_df_determine_trading_state.index = merged_df_determine_trading_state.date

In [541]:
merged_df_determine_trading_state.head()

,date,nbeats_eth_lookback_15_window_5_std_1.25_num_add_dfs_2,tcn_eth_lookback_15_window_5_std_1.25_num_add_dfs_2,nbeats_eth_lookback_30_window_5_std_1.25_num_add_dfs_2,tcn_eth_lookback_30_window_5_std_1.25_num_add_dfs_2,nbeats_eth_lookback_45_window_5_std_1.25_num_add_dfs_2,tcn_eth_lookback_45_window_5_std_1.25_num_add_dfs_2,date_prediction_for,test_model_lookback_1,close,day,month,quarter,day_of_year,year
date,,,,,,,,,,,,,,,
2019-04-02,2019-04-02,160.726046,108.336161,162.136892,4.388280,157.352095,38.469269,2019-04-09,0.0,163.96,2,4,2,092,2019
2019-04-03,2019-04-03,156.490182,43.245200,156.719728,10.110944,144.199683,112.081314,2019-04-10,0.0,161.46,3,4,2,093,2019
2019-04-04,2019-04-04,164.944170,96.866299,150.058959,9.733006,167.251629,24.725918,2019-04-11,0.0,158.05,4,4,2,094,2019
2019-04-05,2019-04-05,169.581252,20.152966,142.574369,2.398744,152.488238,24.184747,2019-04-12,0.0,165.51,5,4,2,095,2019
2019-04-06,2019-04-06,168.788176,25.307259,168.871998,48.561893,178.562880,109.212879,2019-04-13,0.0,165.95,6,4,2,096,2019


In [548]:
merged_df

,date_pred,nbeats_eth_lookback_15_window_5_std_1.25_num_add_dfs_2,tcn_eth_lookback_15_window_5_std_1.25_num_add_dfs_2,nbeats_eth_lookback_30_window_5_std_1.25_num_add_dfs_2,tcn_eth_lookback_30_window_5_std_1.25_num_add_dfs_2,nbeats_eth_lookback_45_window_5_std_1.25_num_add_dfs_2,tcn_eth_lookback_45_window_5_std_1.25_num_add_dfs_2,date_prediction_for,test_model_lookback_1,date_true,close,day,month,quarter,day_of_year,year
0,2019-04-02,160.726046,108.336161,162.136892,4.388280,157.352095,38.469269,2019-04-09,0.0,2019-04-09,176.11,9,4,2,099,2019
1,2019-04-03,156.490182,43.245200,156.719728,10.110944,144.199683,112.081314,2019-04-10,0.0,2019-04-10,177.34,10,4,2,100,2019
2,2019-04-04,164.944170,96.866299,150.058959,9.733006,167.251629,24.725918,2019-04-11,0.0,2019-04-11,165.50,11,4,2,101,2019
3,2019-04-05,169.581252,20.152966,142.574369,2.398744,152.488238,24.184747,2019-04-12,0.0,2019-04-12,164.74,12,4,2,102,2019
4,2019-04-06,168.788176,25.307259,168.871998,48.561893,178.562880,109.212879,2019-04-13,0.0,2019-04-13,164.51,13,4,2,103,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1093,2022-03-30,3306.404151,3023.395365,3727.543705,3117.324561,3220.064806,2726.300653,2022-04-06,0.0,2022-04-06,3167.60,6,4,2,096,2022
1094,2022-03-31,3196.371094,3080.311473,3271.641192,2981.440724,3077.196131,3032.012491,2022-04-07,0.0,2022-04-07,3227.10,7,4,2,097,2022
1095,2022-04-01,3226.660704,3331.778384,3460.590677,3226.893086,3401.522444,3244.867382,2022-04-08,0.0,2022-04-08,3193.40,8,4,2,098,2022
1096,2022-04-02,3535.769389,3175.882685,3255.820443,3200.570144,2786.853463,3207.790335,2022-04-09,0.0,2022-04-09,3258.90,9,4,2,099,2022


In [554]:
merged_df.index = merged_df.date_pred

In [563]:
strategy = 'rf' # mean, rf, lr
verbose = False

mode = 'no_position' # no_position, buy
action_to_take = 'none_to_none' # none_to_none, none_to_buy, buy_to_continue_buy, buy_to_none
has_crossed_mean = False
buy_entry_price = 0
stop_loss_price = 0

PRED_COL = 'close'
window = 5
no_of_std = 1.25
stop_loss_pct=0.15
rolling_mean_col = 'rolling_mean'
bollinger_high_col = 'bollinger_high'
bollinger_low_col = 'bollinger_low'

dollar_amount_buy_lost =  0.0
dollar_amount_buy_won = 0.0
n_buy_lost =  0.0
n_buy_won= 0.0
n_total_days_in_trades = 0.0

buy_has_crossed_mean = 0.0


CHOOSEN_DF = merged_df_determine_trading_state.copy()

rolling_mean = CHOOSEN_DF[PRED_COL].rolling(window).mean()
rolling_std = CHOOSEN_DF[PRED_COL].rolling(window).std()

CHOOSEN_DF[rolling_mean_col] = rolling_mean
CHOOSEN_DF[bollinger_high_col] = rolling_mean + (rolling_std * no_of_std)
CHOOSEN_DF[bollinger_low_col] = rolling_mean - (rolling_std * no_of_std)
CHOOSEN_DF.fillna(method='bfill')


# class vars
price_predictions = 0.0
position_entry_date = None



for row_idx, row in enumerate(CHOOSEN_DF.iterrows()):
    if row_idx <= 7: # need one week
        continue
    
    if row_idx % 10 == 0:
        print(f"{row_idx/len(merged_df_determine_trading_state)*100:.2f}%")
        
    # current stats
    todays_day = row[1].name
    row = CHOOSEN_DF.iloc[row_idx]
    prev_row = CHOOSEN_DF.iloc[row_idx-1:row_idx]
    todays_close_price = row[PRED_COL]
    close_col = PRED_COL
    todays_rolling_mean = row[rolling_mean_col]
    todays_bollinger_high = row[bollinger_high_col]
    todays_bollinger_low = row[bollinger_low_col]
    previous_days_close = prev_row[close_col].values[0]
    previous_days_bollinger_low = prev_row[bollinger_low_col].values[0]
    
    if verbose:
        print(todays_day, 'todays_day')
        print(todays_bollinger_high,' todays_bollinger_high')
        print(todays_close_price, 'todays_close_price')
        print(todays_bollinger_low, 'todays_bollinger_low')
        print(todays_rolling_mean, 'todays_rolling_mean')
        print(previous_days_close,' ,previous_days_close')
        print(previous_days_bollinger_low, 'previous_days_bollinger_low')
        print(mode,' mode')
        print(stop_loss_price, ' stop_loss_price')
        
    # price prediction
    
    if strategy == 'mean':
        # it's a series, so use index
        mean_pred_cols = row.index[~row.index.isin(COLS_TO_EXCLUDE+DAY_PART_COLS)]
        price_prediction = np.mean(row[mean_pred_cols])
        if verbose:
            print(price_prediction, 'price_prediction')


    if strategy == 'rf':
        
        # for training, we need to use the aligned date of the prediction FOR which is in merged_df
        training_df = merged_df[merged_df.index < todays_day]
        testing_df = merged_df[merged_df.index == todays_day]
        
        stacking_training_cols = merged_df.columns[~merged_df.columns.isin(COLS_TO_EXCLUDE)]
        stacked_x_data_train = training_df[stacking_training_cols]# don't include the current row
        stacked_y_data_train = training_df[PRED_COL] # don't include the current row
        
        if verbose:
            print(stacked_x_data_train.shape, 'stacked_x_data_train shape')
            print(stacked_x_data_train.index.min(), 'stacked_x_data_train index min')
            print(stacked_x_data_train.index.max(), 'stacked_x_data_train index max')
            print(stacked_y_data_train.shape, 'stacked_y_data_train.shape')
        rf = RandomForestRegressor(n_estimators=200, n_jobs=-1)
        rf.fit(stacked_x_data_train, stacked_y_data_train)
        # need to predict 
        price_prediction = rf.predict(np.array(testing_df[stacking_training_cols]).reshape(1,-1))
    
    if strategy == 'lr':
        
        # for training, we need to use the aligned date of the prediction FOR which is in merged_df
        training_df = merged_df[merged_df.index < todays_day]
        testing_df = merged_df[merged_df.index == todays_day]
        
        stacking_training_cols = merged_df.columns[~merged_df.columns.isin(COLS_TO_EXCLUDE)]
        stacked_x_data_train = training_df[stacking_training_cols]# don't include the current row
        stacked_y_data_train = training_df[PRED_COL] # don't include the current row
        
        if verbose:
            print(stacked_x_data_train.shape, 'stacked_x_data_train shape')
            print(stacked_x_data_train.index.min(), 'stacked_x_data_train index min')
            print(stacked_x_data_train.index.max(), 'stacked_x_data_train index max')
            print(stacked_y_data_train.shape, 'stacked_y_data_train.shape')
        lr = LinearRegression()
        lr.fit(stacked_x_data_train, stacked_y_data_train)
        # need to predict 
        price_prediction = lr.predict(np.array(testing_df[stacking_training_cols]).reshape(1,-1))
        
    if strategy == 'gb':
        
        # for training, we need to use the aligned date of the prediction FOR which is in merged_df
        training_df = merged_df[merged_df.index < todays_day]
        testing_df = merged_df[merged_df.index == todays_day]
        
        stacking_training_cols = merged_df.columns[~merged_df.columns.isin(COLS_TO_EXCLUDE)]
        stacked_x_data_train = training_df[stacking_training_cols]# don't include the current row
        stacked_y_data_train = training_df[PRED_COL] # don't include the current row
        
        if verbose:
            print(stacked_x_data_train.shape, 'stacked_x_data_train shape')
            print(stacked_x_data_train.index.min(), 'stacked_x_data_train index min')
            print(stacked_x_data_train.index.max(), 'stacked_x_data_train index max')
            print(stacked_y_data_train.shape, 'stacked_y_data_train.shape')
        gb = GradientBoostingRegressor(n_estimators=150)
        gb.fit(stacked_x_data_train, stacked_y_data_train)
        # need to predict 
        price_prediction = gb.predict(np.array(testing_df[stacking_training_cols]).reshape(1,-1))
        

    
    # update stop loss
    if mode == "buy" and (1 - stop_loss_pct) * todays_close_price > stop_loss_price:
        stop_loss_price = (1 - stop_loss_pct) * todays_close_price
        print(f"Updating stop loss to {stop_loss_price}", row)
        
        
    # check if we've previously crossed the mean trailing price
    if mode == "buy" and todays_close_price > todays_rolling_mean:
        buy_has_crossed_mean = 1
        
        # stop loss, get out of buy position
        logger.info(f"self.stop_loss_price = {stop_loss_price}")
    if mode == "buy" and (stop_loss_price > todays_close_price):
        print("stop loss activated for getting out of our buy", row)

        # determine win or loss
        print('determining win or loss')
        mode, buy_entry_price, todays_close_price, position_entry_date, n_buy_lost, n_buy_won, dollar_amount_buy_won, dollar_amount_buy_lost, \
        n_total_days_in_trades= _determine_win_or_loss_amount(row,  mode, buy_entry_price, todays_close_price, position_entry_date, 
                                  n_buy_lost, n_buy_won, dollar_amount_buy_won, dollar_amount_buy_lost, n_total_days_in_trades)
        
        
        
        
        
        # record keeping

        mode = "no_position"
        action_to_take = "buy_to_none"
        buy_has_crossed_mean = 0
        buy_entry_price = 0
        stop_loss_price = 0
        
        # buy -> no_position? no position is below running mean
        # or, if we are above the top band (mean reversion)
    elif mode == "buy" and (
            (
                todays_close_price < todays_rolling_mean
                and buy_has_crossed_mean == 1
            )
            or (todays_close_price > todays_bollinger_high)
            or (todays_close_price < todays_bollinger_low)
            or (todays_rolling_mean < buy_entry_price)
        ):
            
        ## check buy to none
        if (
            (price_prediction < todays_rolling_mean)
            or (price_prediction < buy_entry_price)
            or (todays_rolling_mean < buy_entry_price)
        ):
            # note the order here matters
            # determine win or loss
            print('determining win or loss')
            mode, buy_entry_price, todays_close_price, position_entry_date, n_buy_lost, n_buy_won, dollar_amount_buy_won, dollar_amount_buy_lost, \
            n_total_days_in_trades= _determine_win_or_loss_amount(row,  mode, buy_entry_price, todays_close_price, position_entry_date, 
                              n_buy_lost, n_buy_won, dollar_amount_buy_won, dollar_amount_buy_lost, n_total_days_in_trades)
            # record keeping

            mode = "no_position"
            action_to_take = "buy_to_none"
            buy_has_crossed_mean = 0
            buy_entry_price = 0
            stop_loss_price = 0

            print(f"Exiting buy position . Mode = buy_to_none {row}")
        else:
            print(f"Not exiting buy position {row}")
            action_to_take = "buy_to_continue_buy"
                
                
                
    # buy check with ML model
    elif (
        mode == "no_position"
        and todays_close_price < todays_bollinger_low
        and previous_days_close > previous_days_bollinger_low
    ):
        # check if we should buy

        #self._check_if_we_should_buy(row)

        if price_prediction > todays_rolling_mean:
            # note the order here matters. Want to write logs reflecting the current state
            mode = "buy"
            action_to_take = "none_to_buy"
            buy_entry_price = todays_close_price
            stop_loss_price = todays_close_price * (1 - stop_loss_pct)
            print(row.name, "row.index[0]")
            position_entry_date = pd.to_datetime(row.name).strftime("%Y-%m-%d")
            print('******')
            print(f"ml pred higher than mean taking position {row},price_prediction = {price_prediction} ")
            print('******')
        else:
            print(
                f"self.price_prediction is not higher than the {strategy}. Not going to buy {row}. Price prediction = {price_prediction}"
            )

            action_to_take = "none_to_none"

    elif action_to_take == "none_to_buy":  # not in a position, continue holidng
        action_to_take = "buy_to_continue_buy"
        print(
            f"Taking no action today. Updating none_to_buy to buy_to_continue_buy { row}"
        )

    else:
        if action_to_take == "buy_to_none":
            action_to_take = "none_to_none"
            # self._write_and_print_log_statements(
            #     "Taking no action today. Hit the else statement and updated buy_to_none to none_to_none", row
            # )
        # else:
        #     self._write_and_print_log_statements("Taking no action today. Hit the else statement", row)


self.price_prediction is not higher than the rf. Not going to buy date                                                      2019-04-11
nbeats_eth_lookback_15_window_5_std_1.25_num_add_dfs_2    182.866643
tcn_eth_lookback_15_window_5_std_1.25_num_add_dfs_2        40.237185
nbeats_eth_lookback_30_window_5_std_1.25_num_add_dfs_2    119.877191
tcn_eth_lookback_30_window_5_std_1.25_num_add_dfs_2          2.59137
nbeats_eth_lookback_45_window_5_std_1.25_num_add_dfs_2    176.103259
tcn_eth_lookback_45_window_5_std_1.25_num_add_dfs_2        16.886977
date_prediction_for                                       2019-04-18
test_model_lookback_1                                            0.0
close                                                          165.5
day                                                               11
month                                                              4
quarter                                                            2
day_of_year                          

ValueError: Found array with 0 feature(s) (shape=(1, 0)) while a minimum of 1 is required.

# results


## ETH: 15% stop loss

In [564]:
print(strategy, )
print(dollar_amount_buy_lost, 'dollar_amount_buy_lost')
print(dollar_amount_buy_won, 'dollar_amount_buy_won')
print(dollar_amount_buy_won/dollar_amount_buy_lost, 'win per trade')
print(n_buy_lost, 'n_buy_lost')
print(n_buy_won, 'n_buy_won')
print(n_total_days_in_trades, 'n_total_days_in_trades')
print(n_total_days_in_trades/(n_buy_won+n_buy_lost), 'avg days in trade')
print(n_buy_won/(n_buy_lost+n_buy_won), 'bat rate')

rf
0.0 dollar_amount_buy_lost
8755.539999999999 dollar_amount_buy_won
inf win per trade
0.0 n_buy_lost
28.0 n_buy_won
382.0 n_total_days_in_trades
13.642857142857142 avg days in trade
1.0 bat rate


## ETH: 10% stop loss

In [561]:
print(strategy, )
print(dollar_amount_buy_lost, 'dollar_amount_buy_lost')
print(dollar_amount_buy_won, 'dollar_amount_buy_won')
print(dollar_amount_buy_won/dollar_amount_buy_lost, 'win per trade')
print(n_buy_lost, 'n_buy_lost')
print(n_buy_won, 'n_buy_won')
print(n_total_days_in_trades, 'n_total_days_in_trades')
print(n_total_days_in_trades/(n_buy_won+n_buy_lost), 'avg days in trade')
print(n_buy_won/(n_buy_lost+n_buy_won), 'bat rate')

rf
0.0 dollar_amount_buy_lost
7902.149999999999 dollar_amount_buy_won
inf win per trade
0.0 n_buy_lost
24.0 n_buy_won
337.0 n_total_days_in_trades
14.041666666666666 avg days in trade
1.0 bat rate


## BTC: 20% stop loss

In [504]:
print(strategy, )
print(dollar_amount_buy_lost, 'dollar_amount_buy_lost')
print(dollar_amount_buy_won, 'dollar_amount_buy_won')
print(dollar_amount_buy_won/dollar_amount_buy_lost, 'win per trade')
print(n_buy_lost, 'n_buy_lost')
print(n_buy_won, 'n_buy_won')
print(n_total_days_in_trades, 'n_total_days_in_trades')
print(n_total_days_in_trades/(n_buy_won+n_buy_lost), 'avg days in trade')
print(n_buy_won/(n_buy_lost+n_buy_won), 'bat rate')

rf
35280.67999999999 dollar_amount_buy_lost
33956.39 dollar_amount_buy_won
0.9624641588540812 win per trade
28.0 n_buy_lost
19.0 n_buy_won
233.0 n_total_days_in_trades
4.957446808510638 avg days in trade
0.40425531914893614 bat rate


## BTC: 15% stop loss

In [502]:
print(strategy, )
print(dollar_amount_buy_lost, 'dollar_amount_buy_lost')
print(dollar_amount_buy_won, 'dollar_amount_buy_won')
print(dollar_amount_buy_won/dollar_amount_buy_lost, 'win per trade')
print(n_buy_lost, 'n_buy_lost')
print(n_buy_won, 'n_buy_won')
print(n_total_days_in_trades, 'n_total_days_in_trades')
print(n_total_days_in_trades/(n_buy_won+n_buy_lost), 'avg days in trade')
print(n_buy_won/(n_buy_lost+n_buy_won), 'bat rate')

rf
36096.47 dollar_amount_buy_lost
36210.15000000001 dollar_amount_buy_won
1.0031493384256136 win per trade
26.0 n_buy_lost
23.0 n_buy_won
239.0 n_total_days_in_trades
4.877551020408164 avg days in trade
0.46938775510204084 bat rate


## BTC. 10% stop loss

In [497]:
print(strategy, )
print(dollar_amount_buy_lost, 'dollar_amount_buy_lost')
print(dollar_amount_buy_won, 'dollar_amount_buy_won')
print(dollar_amount_buy_won/dollar_amount_buy_lost, 'win per trade')
print(n_buy_lost, 'n_buy_lost')
print(n_buy_won, 'n_buy_won')
print(n_total_days_in_trades, 'n_total_days_in_trades')
print(n_total_days_in_trades/(n_buy_won+n_buy_lost), 'avg days in trade')
print(n_buy_won/n_buy_lost, 'bat rate')

gb
35931.73 dollar_amount_buy_lost
31775.850000000006 dollar_amount_buy_won
0.8843395516998487 win per trade
30.0 n_buy_lost
19.0 n_buy_won
235.0 n_total_days_in_trades
4.795918367346939 avg days in trade
0.6333333333333333 bat rate


In [493]:
print(strategy, " 200 trees")
print(dollar_amount_buy_lost, 'dollar_amount_buy_lost')
print(dollar_amount_buy_won, 'dollar_amount_buy_won')
print(dollar_amount_buy_won/dollar_amount_buy_lost, 'win per trade')
print(n_buy_lost, 'n_buy_lost')
print(n_buy_won, 'n_buy_won')
print(n_total_days_in_trades, 'n_total_days_in_trades')
print(n_total_days_in_trades/(n_buy_won+n_buy_lost), 'avg days in trade')
print(n_buy_won/n_buy_lost, 'bat rate') # 41%

rf  200 trees
37966.33 dollar_amount_buy_lost
35730.08000000001 dollar_amount_buy_won
0.9410991265155206 win per trade
28.0 n_buy_lost
20.0 n_buy_won
235.0 n_total_days_in_trades
4.895833333333333 avg days in trade
0.7142857142857143 bat rate


In [490]:
print(strategy)
print(dollar_amount_buy_lost, 'dollar_amount_buy_lost')
print(dollar_amount_buy_won, 'dollar_amount_buy_won')
print(dollar_amount_buy_won/dollar_amount_buy_lost, 'win per trade')
print(n_buy_lost, 'n_buy_lost')
print(n_buy_won, 'n_buy_won')
print(n_total_days_in_trades, 'n_total_days_in_trades')
print(n_total_days_in_trades/(n_buy_won+n_buy_lost), 'avg days in trade')
print(n_buy_won/n_buy_lost, 'bat rate')

lr
35552.009999999995 dollar_amount_buy_lost
32858.399999999994 dollar_amount_buy_won
0.9242346635253534 win per trade
26.0 n_buy_lost
11.0 n_buy_won
183.0 n_total_days_in_trades
4.945945945945946 avg days in trade
0.4230769230769231 bat rate


In [488]:
print(strategy)
print(dollar_amount_buy_lost, 'dollar_amount_buy_lost')
print(dollar_amount_buy_won, 'dollar_amount_buy_won')
print(dollar_amount_buy_won/dollar_amount_buy_lost, 'win per trade')
print(n_buy_lost, 'n_buy_lost')
print(n_buy_won, 'n_buy_won')
print(n_total_days_in_trades, 'n_total_days_in_trades')
print(n_total_days_in_trades/(n_buy_won+n_buy_lost), 'avg days in trade')
print(n_buy_won/n_buy_lost, 'bat rate')

rf
35463.200000000004 dollar_amount_buy_lost
36089.04000000001 dollar_amount_buy_won
1.0176475896140225 win per trade
28.0 n_buy_lost
20.0 n_buy_won
235.0 n_total_days_in_trades
4.895833333333333 avg days in trade
0.7142857142857143 bat rate


In [484]:
print(strategy)
print(dollar_amount_buy_lost, 'dollar_amount_buy_lost')
print(dollar_amount_buy_won, 'dollar_amount_buy_won')
print(n_buy_lost, 'n_buy_lost')
print(n_buy_won, 'n_buy_won')
print(n_total_days_in_trades, 'n_total_days_in_trades')
print(n_total_days_in_trades/(n_buy_won+n_buy_lost), 'avg days in trade')
print(n_buy_won/n_buy_lost, 'bat rate')

rf
42444.72 dollar_amount_buy_lost
43059.73000000001 dollar_amount_buy_won
32.0 n_buy_lost
20.0 n_buy_won
232.0 n_total_days_in_trades
4.461538461538462 avg days in trade
0.625 bat rate


In [479]:
print(strategy)
print(dollar_amount_buy_lost, 'dollar_amount_buy_lost')
print(dollar_amount_buy_won, 'dollar_amount_buy_won')
print(n_buy_lost, 'n_buy_lost')
print(n_buy_won, 'n_buy_won')
print(n_total_days_in_trades, 'n_total_days_in_trades')

print(n_total_days_in_trades/(n_buy_won+n_buy_lost), 'avg days in trade')

mean
23940.03 dollar_amount_buy_lost
38239.399999999994 dollar_amount_buy_won
16.0 n_buy_lost
7.0 n_buy_won
135.0 n_total_days_in_trades
5.869565217391305 avg days in trade


In [485]:
7/16

0.4375